# Assignment 5

Deadline: 11.06.2025 12:00 CEST

## Task

Develop an investment strategy for the Swiss equity market, backtest it using the provided datasets (`market_data.parquet`, `jkp_data.parquet`, `spi_index.csv`) and analyze its performance by benchmarking it against the SPI index. Work with the existing code infrastructure (`qpmwp-course`) and extend it by implementing any additional components needed for the strategy. Write a report that presents your methodology and the results.

### Coding (15 points)

- Selection:
  Implement selection item builder functions (via `SelectionItemBuilder`) to filter stocks based on specific criteria (e.g., exclude low-quality or high-volatility stocks).

- Optimization Data & Constraints:
  Implement functions to prepare optimization data (via `OptimizationItemBuilder`), including any econometric or machine learning-based predictions. These functions should also define optimization constraints (e.g., stock, sector, or factor exposure limits).

- Optimization Model:
  If you choose to create a custom optimization model, develop a class inheriting from Optimization (similar to `MeanVariance`, `LeastSquares`, or `BlackLitterman`). Your class should include methods set_objective and solve for defining the objective function and solving the optimization problem.

- Machine Learning Prediction:
  Integrate a machine learning model to estimate inputs for the optimization, such as expected returns or risk. This could include regression, classification, or learning-to-rank models. I suggest you to use the provided jkp_data as features, but you may also create your own (e.g., technical indicators computed on the return or price series).

- Simulation:
  Backtest the strategy and simulate portfolio returns. Account for fixed costs (1% per annum) and variable (transaction) costs (0.2% per rebalancing).


### Report (15 points):

Generate an HTML report with the following sections:

- High-level strategy overview: Describe the investment strategy you developed.

- Detailed explanation of the backtesting steps: Offer a more comprehensive breakdown of the backtesting process, including a description of the models implemented (e.g., details of the machine learning method used).

- Backtesting results:
    
    - Charts: Include visual representations (e.g., cumulative performance charts, rolling 3-year returns, etc.).
    - Descriptive statistics: Present key statistics such as mean, standard deviation, drawdown, turnover, and Sharpe ratio (or any other relevant metric) for the full backtest period as well as for subperiods (e.g., the last 5 years, or during bull vs. bear market phases).
    - Compare your strategy against the SPI index.


In [2]:
# Standard library imports
import os
import sys
import copy
from typing import Optional

# Third party imports
import numpy as np
import pandas as pd

# Add the project root directory to Python path
project_root = os.path.dirname(os.path.dirname(os.getcwd()))   #<Change this path if needed>
src_path = os.path.join(project_root, 'qpmwp-course\\src')    #<Change this path if needed>
sys.path.append(project_root)
sys.path.append(src_path)

# Local modules imports
from helper_functions import load_data_spi, load_pickle
from estimation.covariance import Covariance
from estimation.expected_return import ExpectedReturn
from optimization.optimization import Optimization, Objective, MeanVariance
from optimization.optimization_data import OptimizationData
from optimization.constraints import Constraints
from backtesting.backtest_item_builder_classes import (
    SelectionItemBuilder,
    OptimizationItemBuilder,
)
from backtesting.backtest_item_builder_functions import (
    bibfn_selection_min_volume,
    bibfn_selection_gaps,
    bibfn_return_series,
    bibfn_budget_constraint,
    bibfn_box_constraints,
    bibfn_size_dependent_upper_bounds,
)
from backtesting.backtest_data import BacktestData
from backtesting.backtest_service import BacktestService
from backtesting.backtest import Backtest

In [11]:
# Load data
path_to_data = r'C:\Users\manco\OneDrive\Documentos\UZH\QPMwP\quant-python-last-assing\data' 
# <change this to your path to data>

# Load market and jkp data from parquet files
market_data = pd.read_parquet(path = f'{path_to_data}\market_data.parquet')

# Instantiate the BacktestData class
# and set the market data and jkp data as attributes
data = BacktestData()
data.market_data = market_data
data.bm_series = load_data_spi(path=r'C:\Users\manco\OneDrive\Documentos\UZH\QPMwP\quant-python-last-assing\data')


In [14]:
# Assignment 5 - Data Preprocessing for Swiss Equity Strategy
# Quantitative Portfolio Management with Python

# Standard library imports
import os
import sys
import copy
from typing import Optional

# Third party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add the project root directory to Python path
project_root = os.path.dirname(os.path.dirname(os.getcwd()))   #<Change this path if needed>
src_path = os.path.join(project_root, 'qpmwp-course\\src')    #<Change this path if needed>
sys.path.append(project_root)
sys.path.append(src_path)

# Local modules imports
from helper_functions import load_data_spi, load_pickle
from estimation.covariance import Covariance
from estimation.expected_return import ExpectedReturn
from optimization.optimization import Optimization, Objective, MeanVariance
from optimization.optimization_data import OptimizationData
from optimization.constraints import Constraints
from backtesting.backtest_item_builder_classes import (
    SelectionItemBuilder,
    OptimizationItemBuilder,
)
from backtesting.backtest_item_builder_functions import (
    bibfn_selection_min_volume,
    bibfn_selection_gaps,
    bibfn_return_series,
    bibfn_budget_constraint,
    bibfn_box_constraints,
    bibfn_size_dependent_upper_bounds,
)
from backtesting.backtest_data import BacktestData
from backtesting.backtest_service import BacktestService
from backtesting.backtest import Backtest

# =============================================================================
# 1. LOAD AND INSPECT DATA
# =============================================================================

# Load data
path_to_data = r'C:\Users\manco\OneDrive\Documentos\UZH\QPMwP\quant-python-last-assing\data' 
# <change this to your path to data>

# Load market and jkp data from parquet files
print("Loading market data...")
market_data = pd.read_parquet(path=f'{path_to_data}\\market_data.parquet')

print("Loading JKP factor data...")
try:
    jkp_data = pd.read_parquet(path=f'{path_to_data}\\jkp_data.parquet')
    has_jkp_data = True
    print(f"JKP data loaded successfully with shape: {jkp_data.shape}")
except FileNotFoundError:
    print("JKP data not found - proceeding without factor data")
    jkp_data = None
    has_jkp_data = False

# Instantiate the BacktestData class
data = BacktestData()
data.market_data = market_data
if has_jkp_data:
    data.jkp_data = jkp_data

# Load SPI benchmark
print("Loading SPI benchmark...")
data.bm_series = load_data_spi(path=path_to_data)

print(f"\nData loaded successfully!")
print(f"Market data shape: {market_data.shape}")
if has_jkp_data:
    print(f"JKP data shape: {jkp_data.shape}")
print(f"SPI benchmark shape: {data.bm_series.shape}")

# =============================================================================
# 2. DATA EXPLORATION AND QUALITY ASSESSMENT
# =============================================================================

print("\n" + "="*50)
print("DATA EXPLORATION AND QUALITY ASSESSMENT")
print("="*50)

# Explore market data structure
print("\n--- MARKET DATA STRUCTURE ---")
print("Columns:", market_data.columns.tolist())
print("Index levels:", market_data.index.names)
print("Date range:", market_data.index.get_level_values('date').min(), 
      "to", market_data.index.get_level_values('date').max())
print("Number of unique stocks:", market_data.index.get_level_values('id').nunique())

# Check for missing values in market data
print("\n--- MISSING VALUES IN MARKET DATA ---")
missing_summary = market_data.isnull().sum()
print(missing_summary[missing_summary > 0])

# Analyze return series quality
print("\n--- RETURN SERIES ANALYSIS ---")
return_series = data.get_return_series()
print(f"Return series shape: {return_series.shape}")
print(f"Date range: {return_series.index.min()} to {return_series.index.max()}")

# Check for extreme returns (potential data issues)
extreme_returns = return_series[(return_series > 0.5) | (return_series < -0.5)]
if not extreme_returns.empty:
    print(f"\nFound {len(extreme_returns)} extreme returns (>50% or <-50%):")
    print(extreme_returns.head())

# Volume analysis
print("\n--- VOLUME/LIQUIDITY ANALYSIS ---")
volume_series = data.get_volume_series()
print(f"Volume series shape: {volume_series.shape}")

# Calculate median daily volume by stock
median_volumes = volume_series.median(axis=0).sort_values(ascending=False)
print(f"\nTop 10 stocks by median daily volume:")
print(median_volumes.head(10))
print(f"\nBottom 10 stocks by median daily volume:")
print(median_volumes.tail(10))

# Count zero volume days
zero_volume_days = (volume_series == 0).sum(axis=0).sort_values(ascending=False)
print(f"\nStocks with most zero-volume days:")
print(zero_volume_days.head(10))

# JKP data exploration (if available)
if has_jkp_data:
    print("\n--- JKP FACTOR DATA ANALYSIS ---")
    print("JKP data columns:", jkp_data.columns.tolist())
    print("JKP date range:", jkp_data.index.get_level_values('date').min(), 
          "to", jkp_data.index.get_level_values('date').max())
    
    # Check missing values in JKP data
    jkp_missing = jkp_data.isnull().sum()
    print("\nMissing values in JKP factors:")
    print(jkp_missing[jkp_missing > 0].head(10))

# =============================================================================
# 3. DATA QUALITY FILTERS AND PREPROCESSING
# =============================================================================

print("\n" + "="*50)
print("DATA QUALITY FILTERS AND PREPROCESSING")
print("="*50)

# Create comprehensive data quality filters
def create_data_quality_filters():
    """
    Create a comprehensive set of data quality filters for stock selection
    """
    
    def bibfn_selection_data_quality(bs, rebdate: str, **kwargs) -> pd.DataFrame:
        """
        Comprehensive data quality filter combining multiple criteria
        """
        # Arguments
        min_volume = kwargs.get('min_volume', 500_000)
        min_price = kwargs.get('min_price', 1.0)  # CHF
        max_price = kwargs.get('max_price', 1000.0)  # CHF
        min_trading_days = kwargs.get('min_trading_days', 200)  # out of ~252 trading days
        max_gap_days = kwargs.get('max_gap_days', 10)  # consecutive zero volume days
        lookback_days = kwargs.get('lookback_days', 252)
        
        # Get return and volume data
        return_data = bs.data.get_return_series(
            end_date=rebdate,
            width=lookback_days,
            fillna_value=None
        )
        
        volume_data = bs.data.get_volume_series(
            end_date=rebdate,
            width=lookback_days
        )
        
        # Get price data (reconstruct from returns)
        price_data = (1 + return_data.fillna(0)).cumprod()
        latest_prices = price_data.iloc[-1]
        
        # Initialize results
        stocks = return_data.columns
        results = pd.DataFrame(index=stocks)
        
        # Filter 1: Minimum volume
        median_volume = volume_data.median(axis=0)
        volume_filter = median_volume >= min_volume
        results['volume_filter'] = volume_filter.astype(int)
        
        # Filter 2: Price range
        price_filter = (latest_prices >= min_price) & (latest_prices <= max_price)
        results['price_filter'] = price_filter.astype(int)
        
        # Filter 3: Minimum trading days (non-zero volume)
        trading_days = (volume_data > 0).sum(axis=0)
        trading_filter = trading_days >= min_trading_days
        results['trading_filter'] = trading_filter.astype(int)
        
        # Filter 4: Maximum gap days (consecutive zero volumes)
        def max_consecutive_zeros(series):
            """Calculate maximum consecutive zeros in a series"""
            if series.empty:
                return 0
            zero_groups = (series == 0).astype(int).groupby(series.ne(0).astype(int).cumsum()).sum()
            return zero_groups.max() if not zero_groups.empty else 0
        
        max_gaps = volume_data.apply(max_consecutive_zeros, axis=0)
        gap_filter = max_gaps <= max_gap_days
        results['gap_filter'] = gap_filter.astype(int)
        
        # Filter 5: Data completeness (limit missing returns)
        missing_returns = return_data.isnull().sum(axis=0)
        max_missing = int(0.1 * lookback_days)  # Allow max 10% missing data
        completeness_filter = missing_returns <= max_missing
        results['completeness_filter'] = completeness_filter.astype(int)
        
        # Combine all filters
        all_filters = [
            'volume_filter', 'price_filter', 'trading_filter', 
            'gap_filter', 'completeness_filter'
        ]
        results['binary'] = results[all_filters].min(axis=1)
        
        # Store intermediate values for analysis
        results['median_volume'] = median_volume
        results['latest_price'] = latest_prices
        results['trading_days'] = trading_days
        results['max_gap_days'] = max_gaps
        results['missing_returns'] = missing_returns
        
        return results
    
    return bibfn_selection_data_quality

# Create enhanced selection functions
def create_enhanced_selection_functions():
    """
    Create enhanced selection functions for different market conditions
    """
    
    def bibfn_selection_market_cap_tiers(bs, rebdate: str, **kwargs) -> pd.DataFrame:
        """
        Classify stocks into market cap tiers and apply tier-specific filters
        """
        # Get market cap data
        mcap_data = bs.data.market_data['mktcap']
        mcap_recent = mcap_data[mcap_data.index.get_level_values('date') <= rebdate].groupby('id').last()
        
        # Define market cap tiers (in CHF)
        large_cap_threshold = 10_000_000_000   # 10B CHF
        mid_cap_threshold = 1_000_000_000      # 1B CHF
        small_cap_threshold = 300_000_000      # 300M CHF
        
        results = pd.DataFrame(index=mcap_recent.index)
        results['market_cap'] = mcap_recent
        
        # Classify tiers
        results['tier'] = 'micro'
        results.loc[mcap_recent >= small_cap_threshold, 'tier'] = 'small'
        results.loc[mcap_recent >= mid_cap_threshold, 'tier'] = 'mid'
        results.loc[mcap_recent >= large_cap_threshold, 'tier'] = 'large'
        
        # Apply tier-specific selection (focus on investable universe)
        tier_filter = mcap_recent >= small_cap_threshold  # Exclude micro caps
        results['binary'] = tier_filter.astype(int)
        
        return results
    
    def bibfn_selection_sector_diversification(bs, rebdate: str, **kwargs) -> pd.DataFrame:
        """
        Ensure sector diversification by limiting concentration
        """
        max_sector_weight = kwargs.get('max_sector_weight', 0.4)
        
        # This is a placeholder - in practice you'd need sector classification data
        # For now, we'll create a dummy implementation
        selected_stocks = bs.selection.selected if hasattr(bs, 'selection') else []
        
        results = pd.DataFrame(index=selected_stocks)
        results['binary'] = 1  # Accept all for now
        
        return results
    
    return bibfn_selection_market_cap_tiers, bibfn_selection_sector_diversification

# =============================================================================
# 4. TECHNICAL INDICATORS AND FEATURES
# =============================================================================

def create_technical_features(return_series, volume_series, lookback_window=252):
    """
    Create technical analysis features for stock selection and prediction - OPTIMIZED VERSION
    """
    print("\n--- CREATING TECHNICAL FEATURES (FAST VERSION) ---")
    
    # Initialize features dataframe
    features = pd.DataFrame(index=return_series.index)
    
    # 1. Momentum indicators - MUCH FASTER METHOD
    print("Computing momentum indicators...")
    
    # Use cumulative product method - much faster than apply()
    cum_returns = (1 + return_series.fillna(0)).cumprod()
    
    for months in [1, 3, 6, 12]:
        days = months * 21
        momentum_col = f'momentum_{months}m'
        # Calculate momentum as (current_price / past_price) - 1
        momentum = cum_returns / cum_returns.shift(days) - 1
        features[momentum_col] = momentum.mean(axis=1, skipna=True)
        print(f"  - {momentum_col} completed")
    
    # 2. Volatility indicators - OPTIMIZED
    print("Computing volatility indicators...")
    
    for months in [1, 3, 6]:
        days = months * 21
        vol_col = f'volatility_{months}m'
        # Much faster rolling std calculation
        volatility = return_series.rolling(window=days, min_periods=int(days*0.7)).std() * np.sqrt(252)
        features[vol_col] = volatility.mean(axis=1, skipna=True)
        print(f"  - {vol_col} completed")
    
    # 3. Volume-based indicators - SIMPLIFIED
    print("Computing volume indicators...")
    
    # Simple moving averages - much faster
    volume_ma_short = volume_series.rolling(window=21, min_periods=15).mean()
    volume_ma_long = volume_series.rolling(window=63, min_periods=45).mean()
    
    features['avg_volume'] = volume_ma_short.mean(axis=1, skipna=True)
    # Volume trend as ratio of short to long MA (simpler than polyfit)
    features['volume_trend'] = (volume_ma_short / volume_ma_long - 1).mean(axis=1, skipna=True)
    print("  - Volume indicators completed")
    
    # 4. Market timing indicators - ALREADY FAST
    print("Computing market timing indicators...")
    
    # Market breadth (percentage of stocks with positive returns)
    features['market_breadth'] = (return_series > 0).mean(axis=1, skipna=True)
    
    # Market dispersion (cross-sectional volatility)
    features['market_dispersion'] = return_series.std(axis=1, skipna=True)
    print("  - Market timing indicators completed")
    
    print("✅ All technical features completed!")
    return features

# Create technical features
print("\nCreating technical features...")
technical_features = create_technical_features(
    data.get_return_series(),
    data.get_volume_series()
)

print(f"Technical features created with shape: {technical_features.shape}")
print("Feature columns:", technical_features.columns.tolist())

# =============================================================================
# 5. PREPARE SELECTION ITEM BUILDERS
# =============================================================================

print("\n" + "="*50)
print("PREPARING SELECTION ITEM BUILDERS")
print("="*50)

# Get selection functions
data_quality_filter = create_data_quality_filters()
market_cap_filter, sector_filter = create_enhanced_selection_functions()

# Define comprehensive selection item builders
selection_item_builders = {
    'data_quality': SelectionItemBuilder(
        bibfn=data_quality_filter,
        min_volume=1_000_000,        # 1M CHF daily volume
        min_price=5.0,               # Minimum 5 CHF per share
        max_price=500.0,             # Maximum 500 CHF per share
        min_trading_days=200,        # At least 200 trading days per year
        max_gap_days=5,              # Max 5 consecutive non-trading days
        lookback_days=252            # 1 year lookback
    ),
    
    'market_cap_tiers': SelectionItemBuilder(
        bibfn=market_cap_filter,
    ),
    
    'volume_consistency': SelectionItemBuilder(
        bibfn=bibfn_selection_min_volume,
        width=126,                   # 6 months
        min_volume=500_000,          # 500k CHF
        agg_fn=np.median,
    ),
    
    'trading_gaps': SelectionItemBuilder(
        bibfn=bibfn_selection_gaps,
        width=252,                   # 1 year
        n_days=7,                    # Max 7 consecutive zero-volume days
    ),
}

print("Selection item builders created:")
for name, builder in selection_item_builders.items():
    print(f"  - {name}")

# =============================================================================
# 6. TEST SELECTION FILTERS
# =============================================================================

print("\n" + "="*50)
print("TESTING SELECTION FILTERS")
print("="*50)

# Set up test date
test_dates = data.get_return_series().index[-252:]  # Last year of data
test_date = test_dates[-60].strftime('%Y-%m-%d')    # 60 days from end

print(f"Testing selection filters for date: {test_date}")

# Create a simple backtest service for testing
class SimpleBacktestService:
    def __init__(self, data):
        self.data = data
        self.selection = type('Selection', (), {
            'selected': [],
            'filtered': {},
            'add_filtered': lambda self, name, value: setattr(self, 'filtered', {**getattr(self, 'filtered', {}), name: value})
        })()

# Test the selection filters
bs_test = SimpleBacktestService(data)

print("\nTesting data quality filter...")
try:
    quality_results = data_quality_filter(bs_test, test_date)
    print(f"Data quality filter results shape: {quality_results.shape}")
    print(f"Stocks passing all filters: {quality_results['binary'].sum()}")
    print(f"Filter breakdown:")
    filter_cols = [col for col in quality_results.columns if col.endswith('_filter')]
    for col in filter_cols:
        print(f"  {col}: {quality_results[col].sum()} stocks")
except Exception as e:
    print(f"Error testing data quality filter: {e}")

print("\nTesting market cap filter...")
try:
    mcap_results = market_cap_filter(bs_test, test_date)
    print(f"Market cap filter results shape: {mcap_results.shape}")
    print(f"Stocks by tier:")
    print(mcap_results['tier'].value_counts())
except Exception as e:
    print(f"Error testing market cap filter: {e}")

# =============================================================================
# 7. SUMMARY AND NEXT STEPS
# =============================================================================

print("\n" + "="*50)
print("DATA PREPROCESSING SUMMARY")
print("="*50)

print("\n✅ Data Loading:")
print(f"  - Market data: {market_data.shape[0]:,} rows")
print(f"  - Unique stocks: {market_data.index.get_level_values('id').nunique():,}")
print(f"  - Date range: {market_data.index.get_level_values('date').min()} to {market_data.index.get_level_values('date').max()}")
if has_jkp_data:
    print(f"  - JKP factors: {jkp_data.shape[1]} factors available")

print("\n✅ Data Quality Filters Created:")
print("  - Comprehensive data quality filter (volume, price, completeness)")
print("  - Market cap tier classification")
print("  - Volume consistency filters")
print("  - Trading gap analysis")

print("\n✅ Technical Features:")
print(f"  - {len(technical_features.columns)} technical indicators created")
print("  - Momentum, volatility, volume, and market timing features")

print("\n📋 Next Steps for Strategy Development:")
print("  1. Machine Learning Model Development")
print("     - Use technical features + JKP factors for return prediction")
print("     - Consider ensemble methods (XGBoost, Random Forest)")
print("     - Implement cross-validation with time series splits")
print("\n  2. Portfolio Optimization")
print("     - Implement custom optimization class")
print("     - Add sector/factor exposure constraints")
print("     - Include transaction cost modeling")
print("\n  3. Risk Management")
print("     - Position sizing based on volatility")
print("     - Maximum drawdown controls")
print("     - Sector concentration limits")
print("\n  4. Backtesting Framework")
print("     - Monthly or quarterly rebalancing")
print("     - Out-of-sample testing")
print("     - Performance attribution analysis")

print(f"\n🎯 Ready to proceed with strategy implementation!")
print(f"   Estimated investable universe: ~{quality_results['binary'].sum() if 'quality_results' in locals() else 'TBD'} stocks")
print(f"   Strategy focus: Swiss equity market with systematic factor approach")

Loading market data...
Loading JKP factor data...
JKP data loaded successfully with shape: (130578, 153)
Loading SPI benchmark...

Data loaded successfully!
Market data shape: (1434082, 4)
JKP data shape: (130578, 153)
SPI benchmark shape: (9251,)

DATA EXPLORATION AND QUALITY ASSESSMENT

--- MARKET DATA STRUCTURE ---
Columns: ['price', 'mktcap', 'liquidity', 'sector']
Index levels: ['date', 'id']
Date range: 1985-12-31 00:00:00 to 2024-04-30 00:00:00
Number of unique stocks: 313

--- MISSING VALUES IN MARKET DATA ---
liquidity     105235
sector       1431763
dtype: int64

--- RETURN SERIES ANALYSIS ---
Return series shape: (9977, 313)
Date range: 1986-01-01 00:00:00 to 2024-04-30 00:00:00

Found 9977 extreme returns (>50% or <-50%):
id           1  10  100  101  102  103  104  105  106  107  ...  90  91  92  \
date                                                        ...               
1986-01-01 NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ... NaN NaN NaN   
1986-01-02 NaN NaN 

In [20]:
# Assignment 5 - Enhanced Selection Item Builder Functions
# Implementing sophisticated stock selection criteria

import numpy as np
import pandas as pd
from typing import Optional, List
import warnings
warnings.filterwarnings('ignore')

# =============================================================================
# 1. QUALITY-BASED SELECTION FUNCTIONS
# =============================================================================

def bibfn_selection_comprehensive_quality(bs, rebdate: str, **kwargs) -> pd.DataFrame:
    """
    Comprehensive quality filter combining multiple criteria with sophisticated logic
    """
    # Parameters with more reasonable defaults
    min_volume = kwargs.get('min_volume', 500_000)    # 500K CHF daily (was 1M)
    min_price = kwargs.get('min_price', 1.0)          # 1 CHF minimum (was 5.0)
    max_price = kwargs.get('max_price', 2000.0)       # 2000 CHF maximum (was 500)
    min_market_cap = kwargs.get('min_market_cap', 100_000_000)  # 100M CHF (was 300M)
    min_trading_days = kwargs.get('min_trading_days', 150)      # 150/252 days (was 200)
    max_gap_days = kwargs.get('max_gap_days', 20)               # Max 20 consecutive gaps (was 10)
    lookback_days = kwargs.get('lookback_days', 252)            # 1 year lookback
    volatility_threshold = kwargs.get('volatility_threshold', 1.0)  # 100% annualized (was 60%)
    
    # Get data
    return_data = bs.data.get_return_series(
        end_date=rebdate, width=lookback_days, fillna_value=None
    )
    volume_data = bs.data.get_volume_series(
        end_date=rebdate, width=lookback_days
    )
    
    # Get market cap data
    mcap_data = bs.data.market_data['mktcap']
    mcap_recent = mcap_data[
        mcap_data.index.get_level_values('date') <= rebdate
    ].groupby('id').last()
    
    # Reconstruct price data
    price_data = (1 + return_data.fillna(0)).cumprod()
    latest_prices = price_data.iloc[-1]
    
    # Initialize results
    stocks = return_data.columns
    results = pd.DataFrame(index=stocks)
    
    print(f"Evaluating {len(stocks)} stocks for date {rebdate}")
    
    # === FILTER 1: MARKET CAP ===
    mcap_filter = mcap_recent.reindex(stocks, fill_value=0) >= min_market_cap
    results['market_cap'] = mcap_recent.reindex(stocks, fill_value=0)
    results['mcap_filter'] = mcap_filter.astype(int)
    print(f"  Market cap filter (≥{min_market_cap/1e6:.0f}M): {mcap_filter.sum()}/{len(stocks)} stocks pass")
    
    # === FILTER 2: PRICE RANGE ===
    # More lenient price filter - check for reasonable price levels
    price_filter = (latest_prices >= min_price) & (latest_prices <= max_price) & (latest_prices > 0)
    results['latest_price'] = latest_prices
    results['price_filter'] = price_filter.astype(int)
    print(f"  Price filter ({min_price}-{max_price} CHF): {price_filter.sum()}/{len(stocks)} stocks pass")
    
    # === FILTER 3: LIQUIDITY ===
    median_volume = volume_data.median(axis=0, skipna=True)
    volume_filter = median_volume >= min_volume
    results['median_volume'] = median_volume
    results['volume_filter'] = volume_filter.astype(int)
    print(f"  Volume filter (≥{min_volume/1e6:.1f}M): {volume_filter.sum()}/{len(stocks)} stocks pass")
    
    # === FILTER 4: TRADING CONSISTENCY ===
    # Count non-zero volume days
    trading_days = (volume_data > 0).sum(axis=0)
    trading_filter = trading_days >= min_trading_days
    results['trading_days'] = trading_days
    results['trading_filter'] = trading_filter.astype(int)
    print(f"  Trading consistency (≥{min_trading_days} days): {trading_filter.sum()}/{len(stocks)} stocks pass")
    
    # === FILTER 5: MAXIMUM GAPS ===
    def max_consecutive_zeros(series):
        if series.empty or series.isna().all():
            return float('inf')
        zero_mask = (series == 0) | series.isna()
        if not zero_mask.any():
            return 0
        # Group consecutive zeros and find max length
        groups = zero_mask.astype(int).groupby((~zero_mask).cumsum()).sum()
        return groups.max() if not groups.empty else 0
    
    max_gaps = volume_data.apply(max_consecutive_zeros, axis=0)
    gap_filter = max_gaps <= max_gap_days
    results['max_gap_days'] = max_gaps
    results['gap_filter'] = gap_filter.astype(int)
    print(f"  Gap filter (≤{max_gap_days} consecutive): {gap_filter.sum()}/{len(stocks)} stocks pass")
    
    # === FILTER 6: VOLATILITY SCREEN ===
    # Remove extremely volatile stocks (potential data errors or penny stocks)
    returns_vol = return_data.std(axis=0, skipna=True) * np.sqrt(252)
    volatility_filter = (returns_vol <= volatility_threshold) | returns_vol.isna()
    results['annualized_volatility'] = returns_vol
    results['volatility_filter'] = volatility_filter.astype(int)
    print(f"  Volatility filter (≤{volatility_threshold*100:.0f}%): {volatility_filter.sum()}/{len(stocks)} stocks pass")
    
    # === FILTER 7: DATA COMPLETENESS ===
    missing_returns = return_data.isnull().sum(axis=0)
    max_missing = int(0.25 * lookback_days)  # Allow 25% missing data (was 15%)
    completeness_filter = missing_returns <= max_missing
    results['missing_returns'] = missing_returns
    results['completeness_filter'] = completeness_filter.astype(int)
    print(f"  Completeness filter (≤{max_missing} missing): {completeness_filter.sum()}/{len(stocks)} stocks pass")
    
    # === COMBINE ALL FILTERS ===
    all_filters = [
        'mcap_filter', 'price_filter', 'volume_filter', 
        'trading_filter', 'gap_filter', 'volatility_filter', 'completeness_filter'
    ]
    results['binary'] = results[all_filters].min(axis=1)
    
    final_count = results['binary'].sum()
    print(f"  ✅ FINAL: {final_count}/{len(stocks)} stocks pass ALL filters")
    
    # Debug: Show what's failing
    if final_count < 5:
        print(f"  📊 FILTER BREAKDOWN (showing restrictive filters):")
        for filter_name in all_filters:
            count = results[filter_name].sum()
            if count < len(stocks) * 0.8:  # Show filters that eliminate >20% of stocks
                print(f"    {filter_name}: {count}/{len(stocks)} ({count/len(stocks)*100:.1f}%)")
    
    return results


def bibfn_selection_jkp_factor_quality(bs, rebdate: str, **kwargs) -> pd.DataFrame:
    """
    Filter stocks based on JKP factor data availability and quality
    """
    # Key factors for quality assessment
    key_factors = kwargs.get('key_factors', [
        'be_me', 'market_equity', 'ret_12_1', 'ivol_capm_252d', 
        'qmj', 'ni_me', 'at_me', 'beta_60m'
    ])
    min_factor_coverage = kwargs.get('min_factor_coverage', 0.6)  # 60% of factors must be available
    
    # Get current selection
    current_stocks = bs.selection.selected if hasattr(bs.selection, 'selected') else []
    if not current_stocks:
        # If no previous selection, use all stocks with recent data
        jkp_recent = bs.data.jkp_data[
            bs.data.jkp_data.index.get_level_values('date') <= rebdate
        ].groupby('id').last()
        current_stocks = jkp_recent.index.tolist()
    
    results = pd.DataFrame(index=current_stocks)
    
    if not hasattr(bs.data, 'jkp_data') or bs.data.jkp_data is None:
        print("  ⚠️ No JKP factor data available - skipping factor quality filter")
        results['binary'] = 1
        return results
    
    # Get latest JKP data for each stock
    jkp_recent = bs.data.jkp_data[
        bs.data.jkp_data.index.get_level_values('date') <= rebdate
    ].groupby('id').last()
    
    # Check availability of key factors
    factor_availability = jkp_recent[key_factors].notna().sum(axis=1) / len(key_factors)
    quality_filter = factor_availability >= min_factor_coverage
    
    results['factor_coverage'] = factor_availability.reindex(current_stocks, fill_value=0)
    results['binary'] = quality_filter.reindex(current_stocks, fill_value=0).astype(int)
    
    passed = results['binary'].sum()
    print(f"  JKP factor quality: {passed}/{len(current_stocks)} stocks pass")
    
    return results


# =============================================================================
# 2. MOMENTUM AND TREND-BASED SELECTION
# =============================================================================

def bibfn_selection_momentum_screen(bs, rebdate: str, **kwargs) -> pd.DataFrame:
    """
    Select stocks with positive momentum characteristics
    """
    # Parameters
    momentum_lookback = kwargs.get('momentum_lookback', [63, 126, 252])  # 3, 6, 12 months
    min_positive_momentum = kwargs.get('min_positive_momentum', 2)  # At least 2 periods positive
    exclude_extreme_momentum = kwargs.get('exclude_extreme_momentum', True)
    momentum_threshold = kwargs.get('momentum_threshold', [-0.5, 2.0])  # -50% to +200%
    
    # Get current selection
    current_stocks = bs.selection.selected if hasattr(bs.selection, 'selected') else []
    if not current_stocks:
        return_data = bs.data.get_return_series(end_date=rebdate, width=300)
        current_stocks = return_data.columns.tolist()
    
    results = pd.DataFrame(index=current_stocks)
    
    # Get return data
    return_data = bs.data.get_return_series(
        ids=current_stocks, end_date=rebdate, width=300, fillna_value=0
    )
    
    # Calculate momentum for different periods
    cum_returns = (1 + return_data).cumprod()
    momentum_signals = []
    
    for days in momentum_lookback:
        momentum = cum_returns.iloc[-1] / cum_returns.iloc[-days-1] - 1
        momentum_signals.append(momentum)
        results[f'momentum_{days}d'] = momentum
    
    # Count positive momentum periods
    momentum_df = pd.DataFrame(momentum_signals).T
    positive_count = (momentum_df > 0).sum(axis=1)
    results['positive_momentum_count'] = positive_count
    
    # Apply filters
    momentum_filter = positive_count >= min_positive_momentum
    
    if exclude_extreme_momentum:
        # Exclude stocks with extreme momentum (potential data errors)
        extreme_filter = momentum_df.apply(
            lambda x: ((x >= momentum_threshold[0]) & (x <= momentum_threshold[1])).all(), 
            axis=1
        )
        momentum_filter = momentum_filter & extreme_filter
        results['extreme_filter'] = extreme_filter.astype(int)
    
    results['binary'] = momentum_filter.astype(int)
    
    passed = results['binary'].sum()
    print(f"  Momentum screen: {passed}/{len(current_stocks)} stocks pass")
    
    return results


# =============================================================================
# 3. RISK-BASED SELECTION
# =============================================================================

def bibfn_selection_risk_screen(bs, rebdate: str, **kwargs) -> pd.DataFrame:
    """
    Filter out high-risk stocks based on multiple risk metrics
    """
    # Parameters
    max_volatility = kwargs.get('max_volatility', 0.4)  # 40% annual
    max_beta = kwargs.get('max_beta', 2.0)
    min_observations = kwargs.get('min_observations', 150)  # Min days for risk calc
    lookback_days = kwargs.get('lookback_days', 252)
    
    # Get current selection
    current_stocks = bs.selection.selected if hasattr(bs.selection, 'selected') else []
    if not current_stocks:
        return_data = bs.data.get_return_series(end_date=rebdate, width=lookback_days)
        current_stocks = return_data.columns.tolist()
    
    results = pd.DataFrame(index=current_stocks)
    
    # Get return data
    stock_returns = bs.data.get_return_series(
        ids=current_stocks, end_date=rebdate, width=lookback_days, fillna_value=None
    )
    
    # Get benchmark returns (SPI)
    benchmark_returns = bs.data.bm_series[
        bs.data.bm_series.index <= rebdate
    ].tail(lookback_days)
    
    # Align dates
    common_dates = stock_returns.index.intersection(benchmark_returns.index)
    stock_returns = stock_returns.loc[common_dates]
    benchmark_returns = benchmark_returns.loc[common_dates]
    
    # Calculate risk metrics
    for stock in current_stocks:
        stock_ret = stock_returns[stock].dropna()
        bench_ret = benchmark_returns.reindex(stock_ret.index).dropna()
        
        # Align series
        common_idx = stock_ret.index.intersection(bench_ret.index)
        if len(common_idx) < min_observations:
            results.loc[stock, 'volatility'] = np.nan
            results.loc[stock, 'beta'] = np.nan
            results.loc[stock, 'observations'] = len(common_idx)
            continue
            
        stock_aligned = stock_ret.loc[common_idx]
        bench_aligned = bench_ret.loc[common_idx]
        
        # Calculate volatility
        vol = stock_aligned.std() * np.sqrt(252)
        results.loc[stock, 'volatility'] = vol
        
        # Calculate beta
        covariance = np.cov(stock_aligned, bench_aligned)[0, 1]
        benchmark_var = bench_aligned.var()
        beta = covariance / benchmark_var if benchmark_var > 0 else np.nan
        results.loc[stock, 'beta'] = beta
        results.loc[stock, 'observations'] = len(common_idx)
    
    # Apply filters
    vol_filter = (results['volatility'] <= max_volatility) | results['volatility'].isna()
    beta_filter = (results['beta'] <= max_beta) | results['beta'].isna()
    obs_filter = results['observations'] >= min_observations
    
    results['vol_filter'] = vol_filter.astype(int)
    results['beta_filter'] = beta_filter.astype(int)
    results['obs_filter'] = obs_filter.astype(int)
    
    # Combined filter
    results['binary'] = (vol_filter & beta_filter & obs_filter).astype(int)
    
    passed = results['binary'].sum()
    print(f"  Risk screen: {passed}/{len(current_stocks)} stocks pass")
    
    return results


# =============================================================================
# 4. SECTOR AND SIZE-BASED SELECTION
# =============================================================================

def bibfn_selection_size_tiers(bs, rebdate: str, **kwargs) -> pd.DataFrame:
    """
    Advanced size-based selection with tier preferences
    """
    # Parameters
    large_cap_min = kwargs.get('large_cap_min', 10_000_000_000)   # 10B CHF
    mid_cap_min = kwargs.get('mid_cap_min', 1_000_000_000)        # 1B CHF
    small_cap_min = kwargs.get('small_cap_min', 300_000_000)      # 300M CHF
    
    # Tier preferences (how many stocks from each tier)
    max_large_cap = kwargs.get('max_large_cap', 20)
    max_mid_cap = kwargs.get('max_mid_cap', 30)
    max_small_cap = kwargs.get('max_small_cap', 15)
    
    # Get current selection
    current_stocks = bs.selection.selected if hasattr(bs.selection, 'selected') else []
    if not current_stocks:
        mcap_data = bs.data.market_data['mktcap']
        mcap_recent = mcap_data[
            mcap_data.index.get_level_values('date') <= rebdate
        ].groupby('id').last()
        current_stocks = mcap_recent.index.tolist()
    
    # Get market cap data
    mcap_data = bs.data.market_data['mktcap']
    mcap_recent = mcap_data[
        mcap_data.index.get_level_values('date') <= rebdate
    ].groupby('id').last()
    
    results = pd.DataFrame(index=current_stocks)
    results['market_cap'] = mcap_recent.reindex(current_stocks, fill_value=0)
    
    # Classify size tiers
    results['size_tier'] = 'micro'
    results.loc[results['market_cap'] >= small_cap_min, 'size_tier'] = 'small'
    results.loc[results['market_cap'] >= mid_cap_min, 'size_tier'] = 'mid'
    results.loc[results['market_cap'] >= large_cap_min, 'size_tier'] = 'large'
    
    # Apply tier limits
    results['binary'] = 0
    
    # Select top stocks from each tier by market cap
    for tier, max_count in [('large', max_large_cap), ('mid', max_mid_cap), ('small', max_small_cap)]:
        tier_stocks = results[results['size_tier'] == tier].sort_values('market_cap', ascending=False)
        selected_count = min(len(tier_stocks), max_count)
        if selected_count > 0:
            results.loc[tier_stocks.index[:selected_count], 'binary'] = 1
    
    # Summary
    tier_summary = results.groupby('size_tier')['binary'].sum()
    print(f"  Size tier selection:")
    for tier, count in tier_summary.items():
        print(f"    {tier.capitalize()}: {count} stocks")
    
    total_selected = results['binary'].sum()
    print(f"  Total selected: {total_selected}/{len(current_stocks)} stocks")
    
    return results


# =============================================================================
# 5. COMBINED SELECTION PIPELINE
# =============================================================================

def create_enhanced_selection_builders():
    """
    Create the complete set of enhanced selection item builders with REASONABLE parameters
    """
    
    selection_item_builders = {
        # Core quality filters - RELAXED PARAMETERS
        'comprehensive_quality': SelectionItemBuilder(
            bibfn=bibfn_selection_comprehensive_quality,
            min_volume=300_000,          # 300K CHF daily volume (relaxed from 1M)
            min_price=1.0,               # Min 1 CHF per share (relaxed from 3)
            max_price=2000.0,            # Max 2000 CHF per share (increased from 1000)
            min_market_cap=50_000_000,   # 50M CHF market cap (relaxed from 200M)
            min_trading_days=120,        # At least 120 trading days (relaxed from 180)
            max_gap_days=30,             # Max 30 consecutive gaps (relaxed from 15)
            volatility_threshold=1.2,    # Max 120% annual volatility (relaxed from 80%)
        ),
        
        # JKP factor data quality
        'jkp_factor_quality': SelectionItemBuilder(
            bibfn=bibfn_selection_jkp_factor_quality,
            key_factors=[
                'be_me', 'market_equity', 'ret_12_1', 'ivol_capm_252d',
                'qmj', 'ni_me', 'beta_60m', 'ret_6_1'  # Reduced list of core factors
            ],
            min_factor_coverage=0.4,  # 40% of key factors must be available (relaxed from 50%)
        ),
        
        # Momentum screen - RELAXED
        'momentum_screen': SelectionItemBuilder(
            bibfn=bibfn_selection_momentum_screen,
            momentum_lookback=[63, 126, 252],     # 3, 6, 12 months
            min_positive_momentum=0,              # Allow any momentum (relaxed from 1)
            exclude_extreme_momentum=True,
            momentum_threshold=[-0.9, 5.0],       # Very wide range (relaxed)
        ),
        
        # Risk management - RELAXED
        'risk_screen': SelectionItemBuilder(
            bibfn=bibfn_selection_risk_screen,
            max_volatility=1.5,          # 150% annual volatility (relaxed from 60%)
            max_beta=3.0,                # Beta limit (relaxed from 2.5)
            min_observations=100,        # Min observations (relaxed from 120)
        ),
        
        # Size-based selection - MORE GENEROUS
        'size_tiers': SelectionItemBuilder(
            bibfn=bibfn_selection_size_tiers,
            max_large_cap=20,            # Max 20 large cap stocks
            max_mid_cap=30,              # Max 30 mid cap stocks  
            max_small_cap=15,            # Max 15 small cap stocks
        ),
    }
    
    return selection_item_builders


# =============================================================================
# 6. TESTING THE ENHANCED SELECTION
# =============================================================================

def test_enhanced_selection_fixed(data, test_date='2024-02-07'):
    """
    Test the enhanced selection pipeline - FIXED VERSION
    """
    print("="*60)
    print("TESTING ENHANCED SELECTION PIPELINE - FIXED")
    print("="*60)
    
    # Import functions directly (since they should be in the same environment)
    #from assignment5_selection import create_enhanced_selection_builders
    
    # Create a proper test backtest service
    class TestSelection:
        def __init__(self):
            self.selected = []
            self.filtered = {}
            
        def add_filtered(self, name, value):
            self.filtered[name] = value
            # Update selected list based on binary column
            if 'binary' in value.columns:
                new_selected = value[value['binary'] == 1].index.tolist()
                self.selected = new_selected
                print(f"    → Updated selection: {len(self.selected)} stocks selected")
    
    class TestBacktestService:
        def __init__(self, data):
            self.data = data
            self.selection = TestSelection()
    
    # Create test service
    bs_test = TestBacktestService(data)
    
    # Get enhanced builders
    builders = create_enhanced_selection_builders()
    
    print(f"Testing selection pipeline for date: {test_date}")
    print(f"Available selection filters: {list(builders.keys())}")
    
    # Test each filter sequentially
    results_summary = {}
    total_stocks = data.get_return_series().shape[1]
    
    for filter_name, builder in builders.items():
        print(f"\n--- Testing: {filter_name.upper()} ---")
        try:
            # Run the filter
            filter_result = builder.arguments['bibfn'](bs_test, test_date, **builder.arguments)
            
            # Update selection
            bs_test.selection.add_filtered(filter_name, filter_result)
            
            # Store results
            passed = filter_result['binary'].sum() if 'binary' in filter_result.columns else 0
            total = len(filter_result)
            results_summary[filter_name] = {'passed': passed, 'total': total}
            
            print(f"✅ {filter_name}: {passed}/{total} stocks passed")
            
        except Exception as e:
            print(f"❌ Error in {filter_name}: {str(e)}")
            results_summary[filter_name] = {'passed': 0, 'total': 0, 'error': str(e)}
    
    # Final summary
    print(f"\n{'='*60}")
    print("SELECTION PIPELINE SUMMARY")
    print(f"{'='*60}")
    
    for filter_name, result in results_summary.items():
        if 'error' in result:
            print(f"{filter_name:25}: ERROR - {result['error']}")
        else:
            pct = (result['passed'] / result['total'] * 100) if result['total'] > 0 else 0
            print(f"{filter_name:25}: {result['passed']:3d}/{result['total']:3d} ({pct:5.1f}%)")
    
    final_selection = len(bs_test.selection.selected) if hasattr(bs_test.selection, 'selected') else 0
    print(f"\n🎯 FINAL INVESTABLE UNIVERSE: {final_selection} stocks")
    
    # Show the filter progression
    print(f"\n📊 FILTER PROGRESSION:")
    print(f"Starting universe: {total_stocks} stocks")
    
    current_count = total_stocks
    for filter_name, filter_data in bs_test.selection.filtered.items():
        if 'binary' in filter_data.columns:
            passed = filter_data['binary'].sum()
            print(f"  After {filter_name:20}: {passed:3d} stocks remaining")
            current_count = passed
    
    return bs_test, results_summary

# Test the enhanced selection pipeline with your data
print("🚀 Testing Enhanced Selection Pipeline - FIXED VERSION")
print("="*50)

# Run the test
try:
    test_service, results = test_enhanced_selection_fixed(data, test_date='2024-02-07')
    
    # Get the selection builders for actual use
    enhanced_builders = create_enhanced_selection_builders()
    
    print(f"\n🎯 ANALYSIS OF RESULTS:")
    
    # Show what makes the most impact
    restrictive_filters = []
    for filter_name, result in results.items():
        if 'error' not in result and result['total'] > 0:
            pass_rate = result['passed'] / result['total']
            if pass_rate < 0.5:  # Less than 50% pass rate
                restrictive_filters.append((filter_name, pass_rate, result['passed']))
    
    if restrictive_filters:
        print(f"\n🔍 MOST RESTRICTIVE FILTERS:")
        restrictive_filters.sort(key=lambda x: x[1])  # Sort by pass rate
        for name, rate, count in restrictive_filters:
            print(f"  {name:25}: {rate*100:5.1f}% pass rate ({count} stocks)")
    
    print(f"\n📈 RECOMMENDED INVESTABLE UNIVERSE:")
    final_count = len(test_service.selection.selected) if hasattr(test_service.selection, 'selected') else 0
    print(f"  Final selection: {final_count} stocks")
    print(f"  Universe reduction: {313} → {final_count} ({final_count/313*100:.1f}% of original)")
    
    if final_count > 20:
        print(f"  ✅ Good size for portfolio optimization ({final_count} stocks)")
    elif final_count > 10:
        print(f"  ⚠️  Small but workable universe ({final_count} stocks)")
    else:
        print(f"  ❌ Universe may be too small ({final_count} stocks)")
    
    print(f"\n💡 READY FOR NEXT STEP:")
    print(f"✓ Selection filters working correctly")
    print(f"✓ {len(enhanced_builders)} selection builders ready")
    print(f"✓ Investable universe defined: {final_count} stocks")
    print(f"→ Next: Implement optimization data builders and ML features")

except Exception as e:
    print(f"❌ Test failed: {e}")
    print("Please check that the selection functions are properly loaded")


🚀 Testing Enhanced Selection Pipeline - FIXED VERSION
TESTING ENHANCED SELECTION PIPELINE - FIXED
Testing selection pipeline for date: 2024-02-07
Available selection filters: ['comprehensive_quality', 'jkp_factor_quality', 'momentum_screen', 'risk_screen', 'size_tiers']

--- Testing: COMPREHENSIVE_QUALITY ---
Evaluating 313 stocks for date 2024-02-07
  Market cap filter (≥50M): 288/313 stocks pass
  Price filter (1.0-2000.0 CHF): 214/313 stocks pass
  Volume filter (≥0.3M): 115/313 stocks pass
  Trading consistency (≥120 days): 172/313 stocks pass
  Gap filter (≤30 consecutive): 173/313 stocks pass
  Volatility filter (≤120%): 311/313 stocks pass
  Completeness filter (≤63 missing): 177/313 stocks pass
  ✅ FINAL: 53/313 stocks pass ALL filters
    → Updated selection: 53 stocks selected
✅ comprehensive_quality: 53/313 stocks passed

--- Testing: JKP_FACTOR_QUALITY ---
  JKP factor quality: 53/53 stocks pass
    → Updated selection: 53 stocks selected
✅ jkp_factor_quality: 53/53 stocks 